# Notebook Info

Trying to Identify the best features to use for our algo

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
from library import lib_aws, lib_cleaning
from library.lib_metrics import MultiClassMetrics

## Importing Failures

In [ ]:
# Manually importing data from s3
file_path = "s3://et-oasis/failure-excel/Oasis Complete Failure List 2018-2020_ver6.xlsx"

failures = pd.read_excel(file_path)
# Cleaning WELL NAMES
failures['Well'] = (failures['Well'].str.replace("#", "")  # remove #
                                 .str.replace('\s+', ' ', regex=True)  # remove multiple spaces if present
                                 .str.strip()  # Remove trailing whitespaces
                                 .str.lower()  # lower all character
                                 .str.title()  # Uppercase first letter of each word
                                 .map(lambda x: x[0:-2] + x[-2:].upper()))

failures.rename(columns={'Well': 'NodeID',
                        'LAST OIL': 'Failure Start Date',
                        'FAILURE END DATE': 'Failure End Date'}, inplace=True)
well_list = failures.NodeID.unique().tolist()  # List of wells to use for querying the features and model training
failures.head()

## Importing Data

Import the data depending on how the model is to be trained and the use case:

- For Training: Only wells which have been labelled
- For Hisorical Predictions: The entire dataset
- For Real Time Predictions: Will depend on how the data pipeline has been set up


In [10]:
%%time

# for well_list use only those wells which have been labeled
data_query = """
SELECT
    "NodeID",
    "Date",
    "PPRL",
    "MPRL",
    "FluidLoadonPump",
    "PumpIntakePressure"
FROM xspoc.xdiag
WHERE "NodeID" in {}
ORDER BY "NodeID","Date"
""".format(tuple(well_list))

with lib_aws.PostgresRDS(db='oasis-prod') as engine:
    data = pd.read_sql(data_query, engine, parse_dates=['Date'])

Wall time: 12min 16s


In [394]:
# Failur Info only from wells present in the data
failure_info = failures[failures.NodeID.isin(well_list_features)]
failure_info.reset_index(inplace=True, drop=True)

# info
display(data.head())
print("Failure info in these in these wells")
display(failure_info)
print("Failure Label Distribution")
display(failure_info.Components.value_counts())

,Date,FluidLoadonPump,MPRL,NodeID,PPRL,PumpIntakePressure,Components
0,2019-06-21 15:58:34,3280.0,16811.0,Aagvik 1-35H,27639.0,NaN,Normal
1,2019-06-21 16:25:36,3241.0,16752.0,Aagvik 1-35H,27457.0,NaN,Normal
2,2019-06-21 18:25:16,3330.0,16594.0,Aagvik 1-35H,27448.0,NaN,Normal
3,2019-06-21 18:28:10,3327.0,16595.0,Aagvik 1-35H,27424.0,NaN,Normal
4,2019-06-21 20:25:01,3341.0,16711.0,Aagvik 1-35H,27662.0,NaN,Normal


Failure info in these in these wells


,NodeID,Failure Start Date,LOE START DATE,LOE FINISH DATE,Failure End Date,Components,Job Type,Job Bucket,Primary Symptom,Secondary Symptom,Root Cause
0,Bouvardia Federal 2658 12-12H,2019-08-02 09:57:11,2019-08-13,2019-08-19,2019-08-24 08:31:02,BHA - Seat Nipple,"1-1/2"" PUMP",PUMP,Pump Unseating,NaN,"Erosion Abrasion - Fluids, Solids"
1,Johnson 29-30H,2019-07-08 12:36:03,2019-07-12,2019-07-16,2019-07-16 21:27:22,BHA - Seat Nipple,TUBING LEAK,BHA,Erosion - Fluid,NaN,Undetermined
2,Didrick 4X-27H,2020-01-30 11:53:33,2020-02-10,2020-02-14,2020-02-15 10:16:06,BHA - TAC,TUBING LEAK,BHA,Corrosion,NaN,Improper Chemical Usage
3,Johnsrud 5198 14-18 15TX,2019-08-29 04:00:28,2019-09-05,2019-09-11,2019-09-14 07:01:19,BHA - TAC,BHA - TAC,TUBING,Mechanically Induced Damage,Compression,System Design Needs Improvement
4,Lundeen 4-26H,2019-07-29 12:32:23,2019-08-04,2019-08-12,2019-08-13 09:11:02,BHA - TAC,TUBING LEAK,BHA,Mechanically Induced Damage,Corrosion,Rod Design Needs Improvement
...,...,...,...,...,...,...,...,...,...,...,...
237,Rolfson N 5198 12-17 5T,2019-10-24 13:11:57,2019-10-28,2019-11-04,2019-11-07 07:45:00,Tubing Leak,TUBING LEAK,TUBING,NaN,Mechanically Induced Damage,Tubing Leak - No Hole Found
238,Rj Titus 6093 42-20H,2019-09-13 13:33:45,2019-09-25,2019-09-27,2019-09-29 08:38:56,Tubing Leak,TUBING LEAK,TUBING,Compression,Corrosion,Tubing Leak - No Hole Found
239,Hendricks 5602 43-36 2T,2019-08-11 13:49:43,2019-08-19,2019-08-22,2019-08-26 08:59:47,Tubing Leak,TUBING LEAK,TUBING,Corrosion,NaN,Tubing Leak - No Hole Found
240,Crawford 5493 44-7T,2019-07-28 12:24:42,2019-07-30,2019-08-01,2019-08-01 15:14:46,Tubing Leak,TUBING LEAK,TUBING,Corrosion,Flumping,Tubing Leak - No Hole Found


Failure Label Distribution


Tubing - Body                48
Pump - Plunger               43
Rod - Main Body              25
Pump - Stuck Pump            22
Polish Rod                   19
Tubing Leak                  17
Rod - Pin                    12
Pump - Junked                 9
Rod - 6" Critical Section     8
Pump - Barrel                 8
Rod - Coupling                7
Pump - No-Tap                 6
Tubing - Collar               5
Pump - On - Off Tool          4
BHA - TAC                     3
BHA - Seat Nipple             2
Pump - Standing Valve         2
Pump - Traveling Valve        1
Pump - Valve Rod              1
Name: Components, dtype: int64

## Merging Info

In [395]:
"""
Before analysing the data we need to merge the information
Transfering info from failures to data (copy of features)
Using a for loop -- may not be very efficient
"""

def fill_null(df, chk_col='PPRL', well_col='NodeID', time_col='Date'):
    """
    This function will fill in Null Values on those dates where no datapoints are present
    Helps Show failures where no data was present
    Will have to take this into account when running analysis 
    """
    data_temp = df.copy()
    # Set time col as index if it is not
    if time_col in data_temp.columns:
        data_temp.set_index(time_col, inplace=True)
    
    data_gp = data_temp.groupby(well_col).resample('1D').max()  # Groupby wellname and resample to Day freq
    data_gp.drop(columns=[well_col], inplace=True)  # Drop these columns as they are present in the index
    data_gp.reset_index(inplace=True)  # Get Back WellCol from
    data_null = data_gp[data_gp.loc[:, chk_col].isnull()]  # Get all null values, which need to be added to the main data file
    data_null.reset_index(inplace=True, drop=True)
    data_temp.reset_index(inplace=True)  # get timestamp back in the column for concating
    data_full = pd.concat([data_temp, data_null], axis=0, ignore_index=True)  # concat null and og files
    data_full.sort_values(by=[well_col, time_col], inplace=True)
    data_full.drop_duplicates(subset=[well_col, time_col], inplace=True)
    data_full.reset_index(drop=True, inplace=True)
    
    return data_full

# TODO: transfer_cols only works for multiple columns, get it to work with 1 column
def failure_merge(df, failure_df, transfer_cols):
    """
    Merges the failures info
    :param df: dataframe to which info is being transferred to. (Should have columns "NodeID" and "Date")
    :param failure_df: Failure info data (Should have columns "NodeID", "Start Date" and "End Data")
    :param cols: Columns which need to be transferred
    """
    merged = df.copy()  
    for col in transfer_cols:
        merged[col] = 'Normal'  # for now putting everything as normal (even NAN's)
        
    for i in failure_df.index:
        well = failure_df.loc[i, 'NodeID']
        t_start = failure_df.loc[i, 'Failure Start Date']
        t_end = failure_df.loc[i, 'Failure End Date'] + pd.Timedelta('1 day')  # As we have day based frequency (the times in a day are considered as 00:00:00)
        bool_ = (merged.NodeID == well) & (merged.Date >= t_start) & (merged.Date <= t_end)  # Boolean mask for main data
        merged.loc[bool_, transfer_cols] = failure_df.loc[i, transfer_cols].values
        
    return merged

In [396]:
%%time
data = fill_null(data)  # FIlling in Nan's where data was missing

# Transfer 'Job Bucket' from failure_info to fill_data
transfer_col = ['Components', 'Failure Start Date']
data = failure_merge(data, failure_info, transfer_col)
data.drop(columns='Failure Start Date', inplace=True)

data.head()

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Wall time: 2min 41s


,Components,Date,FluidLoadonPump,MPRL,NodeID,PPRL,PumpIntakePressure
0,Normal,2019-06-21 15:58:34,3280.0,16811.0,Aagvik 1-35H,27639.0,NaN
1,Normal,2019-06-21 16:25:36,3241.0,16752.0,Aagvik 1-35H,27457.0,NaN
2,Normal,2019-06-21 18:25:16,3330.0,16594.0,Aagvik 1-35H,27448.0,NaN
3,Normal,2019-06-21 18:28:10,3327.0,16595.0,Aagvik 1-35H,27424.0,NaN
4,Normal,2019-06-21 20:25:01,3341.0,16711.0,Aagvik 1-35H,27662.0,NaN


## Feature Engg

Depending on What features we and labels we want to use for our model, we can use the functions

`get_agg()`:

    - For now only gives us moving averages
    - Can modify it to give other aggregate functions
    
`create_prediction_zones()`:
    
    - Will create new classes depending on what windows we choose for failures
  
**Note: Both these fucntions will give out separate dataframes/series and will have to be merged accordingly**


In [589]:
"""
Helper Functions
"""

def get_agg(df, freq, time_col='Date', well_col = 'NodeID'):
    
    frames = []
    
    for well in df[well_col].unique():
        temp_df = df[df[well_col] == well].copy()
        temp_df.set_index(time_col, inplace=True)
        temp_df = temp_df.rolling(freq).mean()
        temp_df = temp_df.add_prefix(freq+'_')
        temp_df[well_col] = well
        temp_df.reset_index(inplace=True)
        frames.append(temp_df)
        
    rolled_df = pd.concat(frames)
    rolled_df.reset_index(inplace=True, drop=True)
    
    return rolled_df


def create_prediction_zones(df, fail_col, prediction_zone_dict):
    """
    Depending on the prediction_zone_dict will create predictions zones for failures 
    in the Failure column.
    :param df: The dataframe to extract it from
    :param fail_col: Failure column to use from the dataframe
    :param prediction_zone_dict: A dict with timedeltas for each type of Failure in fail_col
    :return Will return a Series or an Array of these Prediction Zones
    """
    
    test_data = df[['NodeID', 'Date', fail_col]].copy()
    fail_zones = test_data[fail_col]  # fail_zones will be initialized as a copy of the fail col
    
    # Getting start of predictions from fail col
    fail_dates = test_data[test_data[fail_col] != 'Normal']  # everthing other than normal is considered as a prediction
    fail_start = fail_dates[fail_dates.Date.diff().abs().fillna(pd.Timedelta('10D')) > pd.Timedelta('1d 12H')]
    fail_start.reset_index(inplace=True, drop=True)
    
    # Adding zones by iterating over each prediction start date
    for i in fail_start.index:
        temp_well = fail_start.loc[i, 'NodeID']  # well name
        zone_end_date = fail_start.loc[i, 'Date']  # prediction start date
        fail = fail_start.loc[i, fail_col]  # actual prediction class
        zone_delta = pd.Timedelta(prediction_zone_dict[fail])  # delta to subtract from the dictionary
        zone_start_date = zone_end_date - zone_delta

        bool_ = (test_data.NodeID == temp_well) & (test_data.Date < zone_end_date) & (test_data.Date >= zone_start_date)
        fail_zones[bool_] = 'fz_' + fail
        
    return fail_zones

Say we want to use rolling averages with a frequency of 7 days for our features and a constant 10 day window for our failures. Follow the next few sections to see how it will be done

In [ ]:
%%time
# 7 day rolling averages
avg_data = get_agg(df=data, freq='15D')  

# Merge it with the original data 
# and use only those columns which will be of use
# While working with large datasets try optmizing the copies of dataframes you create
# May not even have to merge it

full_data = data.set_index(['NodeID', 'Date']).merge(avg_data.set_index(['NodeID', 'Date']), 
                                                     left_index=True,
                                                     right_index=True).reset_index()



In [ ]:
# Drop Columns that we dont need
cols_drop = [
    'PPRL',
    'MPRL',
    'PumpIntakePressure',
    'FluidLoadonPump',
]


full_data.drop(columns=cols_drop, inplace=True)


full_data.head()

In [ ]:
full_data.isnull().sum(axis=0)/len(full_data) * 100

In [593]:
# pred zone dict
# manual
# pred_zone_dict = {
#     'PUMP': '15 days',
#     'ROD': '15 days',
#     'TUBING': '15 days',
#     'BHA': '15 days'
# }

# automated all failures will have the same failure window
fail_window = '15days'
fail_labels = full_data.Components.unique().tolist()
fail_labels.remove('Normal')
pred_zone_dict = {x: fail_window for x in fail_labels}

In [ ]:
# Create pred windows
# Note:  The output of the fucntion will be a pandas Series
full_data['Label'] = create_prediction_zones(df=full_data, 
                                             fail_col='Components', 
                                             prediction_zone_dict=pred_zone_dict)

Some Assumtions we are going to make:
- Drop Nan values
- Remove the actual failures as classes and only use windows

In [ ]:
full_data.dropna(inplace=True)

class_drop = fail_labels  # just for now dont need to use it
full_data = full_data[~full_data.Label.isin(class_drop)]
full_data.Label = full_data.Label.str.replace('fz_', '').str.strip()
full_data.reset_index(inplace=True, drop=True)

# Feature Analysis

In [ ]:
import seaborn as sns
from sklearn.preprocessing import  LabelEncoder
fdt = full_data.copy()
fdt.drop(['NodeID','Date','Components'],axis=1,inplace=True)

fdt['Label'] = fdt[['Label']].apply(LabelEncoder().fit_transform)
plt.figure(figsize=(8,6))
sns.heatmap(fdt.corr(),annot=True)

In [ ]:
class analysis:
  def __init__(self,a,b):
    self.x = a
    self.Y = b

  def e_tree(self):
    X = self.x #independent columns
    y = self.Y    #target column 
    from sklearn.ensemble import ExtraTreesClassifier
    model = ExtraTreesClassifier()
    model.fit(X,y)
    print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
    #plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    feat_importances.nlargest(13).plot(kind='barh')
    plt.show()

In [ ]:
an = analysis(fdt.drop('Label',axis=1),fdt['Label'])
an.e_tree()

In [ ]:
X = full_data.drop(['NodeID', 'Date', 'Components','Label'],axis=1)
Y = full_data.Label

print("Features")
display(X.head())

print("Classes Being Predicted")
display(Y.value_counts())

# Testing Algos

In [ ]:
# Imports
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
"""
Model 1
Random Forest classifier
"""

def build_rfc_model ():
    """
    Define A Random Forrest Classifier Model
    :return: RFC Model
    """
    scaler = StandardScaler() # Define Scaler
 
    # RFC Params
    rfc_params = {
        'n_estimators': 100,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'class_weight': 'balanced',
        'verbose': 0,
        'max_features': 'auto',
        'max_depth': None,
    }
    
    # RFC Classifier
    rfc = RandomForestClassifier(**rfc_params)
    
    #
    model = Pipeline([
        ('scaler', scaler),
        ('rfc', rfc)
    ])

    return model
######################
rfc_model = build_rfc_model()
MultiClassMetrics.baseline_metrics(X, Y, rfc_model)

In [ ]:
# cv_rfc = MultiClassMetrics.cv_validation(X, Y, rfc_model)
# print("CV Metrics")
# display(cv_rfc)

In [ ]:
kf_rfc = MultiClassMetrics.kfold_validation(X, Y, rfc_model)
print("Kfold Metrics")
display(kf_rfc)

In [ ]:
"""
Model 7
ExtraTreesClassifier
"""
from sklearn.ensemble import ExtraTreesClassifier


def build_et_model():

    scaler = StandardScaler()

    et_params ={ 'class_weight': 'balanced', 
                'verbose': 1
    }

    et = ExtraTreesClassifier(**et_params)

    model = Pipeline([
        ('scaler', scaler),
        ('et', et)
    ])

    return model

###########################################
et_model = build_et_model()
MultiClassMetrics.baseline_metrics(X, Y, et_model)




In [ ]:
kf_rfc = MultiClassMetrics.kfold_validation(X, Y, et_model)
print("Kfold Metrics")
display(kf_rfc)

In [ ]:
"""
Model 6
XGBOOST
"""
from xgboost import XGBClassifier


def build_xg_model():

    scaler = StandardScaler()

    xg_params ={ 'objective': 'multi:softmax', #multi:softprob
                'num_class': full_data['Components'].nunique()
    }

    xg = XGBClassifier(**xg_params)

    model = Pipeline([
        ('scaler', scaler),
        ('xg', xg)
    ])

    return model

##########################################################################

xg_model = build_xg_model()
MultiClassMetrics.baseline_metrics(X, Y, xg_model)


In [ ]:
kf_rfc = MultiClassMetrics.kfold_validation(X, Y, xg_model)
print("Kfold Metrics")
display(kf_rfc)